In [123]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers import Trainer, TrainingArguments
import torch
import random
import numpy as np
import pandas as pd
import spacy
import re
import stopwordsiso as stopwords
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset


In [124]:
data = pd.read_json('../task_1/data/fragments_classification.jsonl', lines=True)
data.tail()

,text,label
795,@USER Wszystkiego najlepszego z okazji urodzin...,"[[5, 29, wzmocnienie]]"
796,"@USER widzę, że pewne tweety działają jak magn...",[]
797,"@USER @USER Chociaż futro ma z jenota,\nTo nie...","[[43, 52, odwrócenie], [55, 67, wzmocnienie], ..."
798,@USER Ty aby nie zacząleś ćpać przez wydumane ...,"[[13, 25, odwrócenie], [37, 54, wzmocnienie]]"
799,"@USER och, to też mój superman w kuchni. Jak c...",[]


In [125]:
def adjust_labels_to_words(row):
    text = row['text']
    labels = row['label']
    adjusted_labels = []
    
    for start_idx, end_idx, label in labels:
        while start_idx > 0 and text[start_idx-1].isalnum():
            start_idx -= 1
        
        while end_idx < len(text) and text[end_idx].isalnum():
            end_idx += 1
        
        adjusted_labels.append((start_idx, end_idx, label))
    
    return adjusted_labels

nlp = spacy.load("pl_core_news_sm")
def preprocess_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower().strip()
    doc = nlp(text)

    return [token.lemma_ for token in doc]


def get_labels(row):
    text = row['text']
    return [(preprocess_text(text[start_idx : end_idx]), label) for start_idx, end_idx, label in row['label']]

In [ ]:
def mask_labels(df, label_to_increase='osłabienie'):
    new_rows = []
    for index, row in df.iterrows():
        original_text = row['text']
        original_labels = row['label']
        masked_text = original_text
        
        for label in original_labels:
            start, end, lbl = label
            if lbl == label_to_increase:
                masked_text = masked_text[:start] + "[MASK]" + masked_text[end:]
        
                new_rows.append({'text': masked_text, 'label': original_labels})
    
    augmented_df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    return augmented_df

augmented_df = mask_labels(data, label_to_increase='osłabienie')


In [130]:
augmented_df.tail()

,text,label
892,@USER [MASK] nie było ciekawszych? :) [MASK]że...,"[[6, 10, osłabienie], [11, 19, odwrócenie], [2..."
893,@USER [MASK] już nie.kompromitujcie się dalej,"[[6, 10, osłabienie], [15, 33, odwrócenie]]"
894,@USER @USER @USER no to można spróbować innych...,"[[57, 61, osłabienie]]"
895,"@USER Na Żoliborzu \""pożar w burdelu\"" sondaż...","[[60, 69, osłabienie], [70, 80, wzmocnienie]]"
896,"@USER o widzisz, to nawet nie wiedziałam bo to...","[[26, 40, odwrócenie], [52, 58, osłabienie]]"


In [131]:
def adjust_labels_to_words(row):
    text = row['text']
    labels = row['label']
    adjusted_labels = []
    
    for start_idx, end_idx, label in labels:
        while start_idx > 0 and text[start_idx-1].isalnum():
            start_idx -= 1
        
        while end_idx < len(text) and text[end_idx].isalnum():
            end_idx += 1
        
        adjusted_labels.append((start_idx, end_idx, label))
    
    return adjusted_labels


def preprocess_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower().strip()
    doc = nlp(text)

    return [token.lemma_ for token in doc]


def get_labels(row):
    text = row['text']
    return [(preprocess_text(text[start_idx : end_idx]), label) for start_idx, end_idx, label in row['label']]

In [132]:
augmented_df['label'] = augmented_df.apply(adjust_labels_to_words, axis = 1)
augmented_df['tokens'] = augmented_df['text'].apply(preprocess_text)
augmented_df['labels2'] = augmented_df.apply(get_labels, axis = 1)

augmented_df.head()

,text,label,tokens,labels2
0,Nie uzna gola. Robben był kilka metrów w polu ...,"[(0, 8, odwrócenie)]","[nie, uzny, gol, robben, być, kilka, metr, w, ...","[([nie, uzny], odwrócenie)]"
1,@USER No właśnie o tym jest ten tweet 😄,[],"[no, właśnie, o, to, być, ten, tweet]",[]
2,@USER @USER Widać chcą wiecej polskich mord go...,"[(23, 38, wzmocnienie)]","[widać, chcieć, wiecej, polski, mord, gotujacy...","[([wiecej, polski], wzmocnienie)]"
3,"Idę spać bo padam na twarz, w końcu w domuuuu",[],"[iść, spać, bo, padać, na, twarz, w, koniec, w...",[]
4,@USER Tak się poznałam z moim chłopakiem 😂 cza...,[],"[tak, się, poznać być, z, mój, chłopak, , cza...",[]


In [136]:
augmented_df.tail()

,text,label,tokens,labels2
892,@USER [MASK] nie było ciekawszych? :) [MASK]że...,"[(6, 11, osłabienie), (7, 21, odwrócenie), (17...","[mask, nie, być, ciekawszy, , maskże, ten, ag...","[([mask], osłabienie), ([mask, nie, być], odwr..."
893,@USER [MASK] już nie.kompromitujcie się dalej,"[(6, 11, osłabienie), (13, 35, odwrócenie)]","[mask, już, niekompromitujcie, się, daleko]","[([mask], osłabienie), ([już, niekompromitujci..."
894,@USER @USER @USER no to można spróbować innych...,"[(57, 62, osłabienie)]","[no, to, można, spróbować, inny, wariantówmask...","[([mask], osłabienie)]"
895,"@USER Na Żoliborzu \""pożar w burdelu\"" sondaż...","[(60, 77, osłabienie), (67, 80, wzmocnienie)]","[na, żoliborz, , pożar, w, burdel, sondaż, w,...","[([maskkłamczuch], osłabienie), ([kłamczuch], ..."
896,"@USER o widzisz, to nawet nie wiedziałam bo to...","[(26, 40, odwrócenie), (52, 58, osłabienie)]","[o, widzieć, to, nawet, nie, wiedzieć być, bo,...","[([nie, wiedzieć być], odwrócenie), ([mask], o..."
